Note: One of the goals of ExploSig is enabling analysis of mutational signatures and clinical/molecular datasets simultaneously. The ExploSig Browser (http://explosig.lrgr.io) provides tools for initial exploration and simple statistics with stratification and filtering of samples, but there are currently no statistical tests that can be performed within the tool. The NTHL1 case study we describe was performed with the ExploSig Browser. This notebook reproduces the steps of that visual exploration programmatically with processed data obtained using the ExploSig Connect package, enabling further quantitative analysis.

In [80]:
import numpy as np
from scipy import stats

In [3]:
from explosig_connect import connect

projects = ['TCGA-BRCA_BRCA_mc3.v0.2.8.WXS']
signatures = ["COSMIC %d" % x for x in [1, 2, 3, 5, 6, 8, 13, 17, 18, 20, 26, 30]]
genes = ['NTHL1', 'BRCA1', 'BRCA2']

### Get processed data from ExploSig Server

In [20]:
conn = connect(empty=True, how=None)
exps_df = conn.get_exposures(projects, signatures, 'SBS')
gene_exp_df = conn.get_gene_expression_data(genes, projects)
gene_mut_df = conn.get_gene_mutation_data(genes, projects)
gene_cna_df = conn.get_copy_number_data(genes, projects)

### Filter and stratify samples

In [50]:
print(f"{exps_df.shape[0]} samples")

1020 samples


Filter to restrict to samples with wildtype BRCA1 and wildtype BRCA2

In [56]:
BRCA_mut_samples = gene_mut_df.loc[(gene_mut_df["BRCA1"] != "None") | (gene_mut_df["BRCA2"] != "None")].index.values.tolist()
BRCA_wt_samples = gene_mut_df.loc[(gene_mut_df["BRCA1"] == "None") & (gene_mut_df["BRCA2"] == "None")].index.values.tolist()

print(f"{len(BRCA_mut_samples)} samples have mut BRCA1 and/or mut BRCA2")
print(f"{len(BRCA_wt_samples)} samples have wildtype BRCA1 and wildtype BRCA2")

57 samples have mut BRCA1 and/or mut BRCA2
963 samples have wildtype BRCA1 and wildtype BRCA2


In [57]:
BRCA_mut_exps_df = exps_df.loc[BRCA_mut_samples, :]
BRCA_wt_exps_df = exps_df.loc[BRCA_wt_samples, :]

print(f"{BRCA_mut_exps_df.shape[0]} samples have mut BRCA1 and/or mut BRCA2, and also have exposures data")
print(f"{BRCA_wt_exps_df.shape[0]} samples have wildtype BRCA1 and wildtype BRCA2, and also have exposures data")

57 samples have mut BRCA1 and/or mut BRCA2, and also have exposures data
963 samples have wildtype BRCA1 and wildtype BRCA2, and also have exposures data


Distribution of COSMIC 3 exposure values for BRCA1/2 mutant samples

In [53]:
BRCA_mut_cosmic_3_exposure_stats = BRCA_mut_exps_df.describe()["COSMIC 3"]
BRCA_mut_cosmic_3_exposure_stats.to_frame()

,COSMIC 3
count,57.000000
mean,39.197164
std,40.932146
min,0.000000
25%,4.984054
50%,33.650903
75%,60.120945
max,156.641167


Distribution of COSMIC 3 exposure values for BRCA1/2 wildtype samples 

In [54]:
BRCA_wt_cosmic_3_exposure_stats = BRCA_wt_exps_df.describe()["COSMIC 3"]
BRCA_wt_cosmic_3_exposure_stats.to_frame()

,COSMIC 3
count,9.630000e+02
mean,1.286567e+01
std,2.496890e+01
min,0.000000e+00
25%,6.612271e-17
50%,2.273541e+00
75%,1.350781e+01
max,1.869085e+02


### Stratification of BRCA1/2 wildtype samples by NTHL1 gene expression level

In [77]:
BRCA_wt_gene_exp_df = gene_exp_df.loc[BRCA_wt_samples, :]
BRCA_wt_NTHL1_gene_exp_sample_groups = BRCA_wt_gene_exp_df.groupby("NTHL1").groups
BRCA_wt_NTHL1_overexp_cosmic_3_df = BRCA_wt_exps_df.loc[BRCA_wt_NTHL1_gene_exp_sample_groups["Over"].values.tolist(), :]
BRCA_wt_NTHL1_nondiffexp_cosmic_3_df = BRCA_wt_exps_df.loc[BRCA_wt_NTHL1_gene_exp_sample_groups["Not differentially expressed"].values.tolist(), :]

print(f"{BRCA_wt_NTHL1_overexp_cosmic_3_df.shape[0]} BRCA1/2 wildtype samples with overexpression of NTHL1")
print(f"{BRCA_wt_NTHL1_nondiffexp_cosmic_3_df.shape[0]} BRCA1/2 wildtype samples with non-differential expression of NTHL1")

92 BRCA1/2 wildtype samples with overexpression of NTHL1
871 BRCA1/2 wildtype samples with non-differential expression of NTHL1


Distribution of COSMIC 3 exposure values for BRCA1/2 wildtype AND overexpressed NTHL1 samples

In [74]:
BRCA_wt_NTHL1_overexp_cosmic_3_exposure_stats = BRCA_wt_NTHL1_overexp_cosmic_3_df.describe()["COSMIC 3"]
BRCA_wt_NTHL1_overexp_cosmic_3_exposure_stats.to_frame()

,COSMIC 3
count,9.200000e+01
mean,1.285349e+01
std,2.566682e+01
min,0.000000e+00
25%,2.007702e-16
50%,2.450169e+00
75%,1.251392e+01
max,1.310209e+02


Distribution of COSMIC 3 exposure values for BRCA1/2 wildtype AND non-differentially-expressed NTHL1 samples

In [78]:
BRCA_wt_NTHL1_nondiffexp_cosmic_3_exposure_stats = BRCA_wt_NTHL1_nondiffexp_cosmic_3_df.describe()["COSMIC 3"]
BRCA_wt_NTHL1_nondiffexp_cosmic_3_exposure_stats.to_frame()

,COSMIC 3
count,8.710000e+02
mean,1.286696e+01
std,2.490916e+01
min,0.000000e+00
25%,5.757342e-17
50%,2.245123e+00
75%,1.354028e+01
max,1.869085e+02


T-test to compare the means of these two groups

In [92]:
NTHL1_gene_exp_ttest = stats.ttest_ind(BRCA_wt_NTHL1_overexp_cosmic_3_df["COSMIC 3"].values, BRCA_wt_NTHL1_nondiffexp_cosmic_3_df["COSMIC 3"].values, equal_var=False)
print(f"p-value: {NTHL1_gene_exp_ttest.pvalue}")

p-value: 0.9961772499647088


### Stratification of BRCA1/2 wildtype samples by NTHL1 copy number status